# Offer Search Engine

Hi! Welcome to the tutorial walkthrough for creating an offer retrieval pipeline given offers by different brands and retailers on variety of products.

We are also given the type of products sold by different brands in a different table. So a user can also query the type of products and check whether there are any offers on those categories.

The pipeline includes feature extraction using Sentence Transformers, indexing using Vector Store and Semantic Search with Approximate Nearest Neighbor tool for fast and efficient retrieval.

Let's dive in!

First let's install necessary dependencies

In [34]:
!pip install sentence-transformers faiss-cpu ftfy -q

In [2]:
import pandas as pd
import sqlite3
import numpy as np
from numpy.linalg import norm
import ftfy
import html
import faiss

### Getting the data

We will download the data stored in 3 different tables and load them using pandas dataframes for fast and easier processing

In [35]:
!wget https://raw.githubusercontent.com/gargsid/receipt_data_prediction_app/main/assets/data/brand_category.csv
!wget https://raw.githubusercontent.com/gargsid/receipt_data_prediction_app/main/assets/data/categories.csv
!wget https://raw.githubusercontent.com/gargsid/receipt_data_prediction_app/main/assets/data/offer_retailer.csv

--2023-12-22 17:33:25--  https://raw.githubusercontent.com/gargsid/receipt_data_prediction_app/main/assets/data/brand_category.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 270151 (264K) [text/plain]
Saving to: ‘brand_category.csv.2’

brand_category.csv. 100%[===================>] 263.82K  --.-KB/s    in 0.1s    

2023-12-22 17:33:25 (2.38 MB/s) - ‘brand_category.csv.2’ saved [270151/270151]

--2023-12-22 17:33:25--  https://raw.githubusercontent.com/gargsid/receipt_data_prediction_app/main/assets/data/categories.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request s

In [36]:
brands_df = pd.read_csv('brand_category.csv')
categories_df = pd.read_csv('categories.csv')
retailers_df = pd.read_csv('offer_retailer.csv')

### Data Analysis

Let's look at few rows from all the three dataframes.

First table shows different brands and types of products sold by them.

In [25]:
brands_df.head()

,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS
0,CASEYS GEN STORE,Tobacco Products,2950931
1,CASEYS GEN STORE,Mature,2859240
2,EQUATE,Hair Removal,893268
3,PALMOLIVE,Bath & Body,542562
4,DAWN,Bath & Body,301844


Second we have different type of product categories heirarchy.

In [26]:
categories_df.head()

,CATEGORY_ID,PRODUCT_CATEGORY,IS_CHILD_CATEGORY_TO
0,1f7d2fa7-a1d7-4969-aaf4-1244f232c175,Red Pasta Sauce,Pasta Sauce
1,3e48a9b3-1ab2-4f2d-867d-4a30828afeab,Alfredo & White Pasta Sauce,Pasta Sauce
2,09f3decc-aa93-460d-936c-0ddf06b055a3,Cooking & Baking,Pantry
3,12a89b18-4c01-4048-94b2-0705e0a45f6b,Packaged Seafood,Pantry
4,2caa015a-ca32-4456-a086-621446238783,Feminine Hygeine,Health & Wellness


Finally, we have list of offers by brands and retailers.

In [27]:
retailers_df.head()

,OFFER,RETAILER,BRAND
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB
1,"Beyond Meat® Plant-Based products, spend $25",NaN,BEYOND MEAT
2,Good Humor Viennetta Frozen Vanilla Cake,NaN,GOOD HUMOR
3,"Butterball, select varieties, spend $10 at Dil...",DILLONS FOOD STORE,BUTTERBALL
4,"GATORADE® Fast Twitch®, 12-ounce 12 pack, at A...",AMAZON,GATORADE


### Entity Linking

The user can query retailers, brand and product category for different type of products. Offers are linked to brands and retailers in the `retailers_df` table.

However, to link the offers with product categories also, we joined the `retailers_df` and `brands_df` using left inner join so that all the entries of retailers_df are kept (as we do not want to lose any offers).

We joined on the `PRODUCT_CATEGORY` column. Furthermore, there are multiple product categories for the same product so we aggregate those into single entry separated by columns.

We can see the few entries of the final `offers_df`

In [37]:
brands_df = brands_df.rename(columns={'BRAND_BELONGS_TO_CATEGORY': 'PRODUCT_CATEGORY'})
brands_df = brands_df.drop('RECEIPTS', axis=1)

categories_df = categories_df.drop('CATEGORY_ID', axis=1)

offers_df = pd.merge(retailers_df, brands_df, on='BRAND', how='left')
offers_df = offers_df.fillna('')

# for a singe entry (offer, retailer, brand) there could be multilpe product categories -> we aggregate those categories
offers_df = offers_df.groupby(['OFFER', 'RETAILER', 'BRAND'])['PRODUCT_CATEGORY'].apply(', '.join).reset_index()

offers_df.to_csv('offers_df.csv')

In [38]:
offers_df.head()

,OFFER,RETAILER,BRAND,PRODUCT_CATEGORY
0,12 Pack OR 2 Liter AND Whole Pizza Pie at Casey's,CASEYS GENERAL STORE,CASEYS GENERAL STORE,"Cooking & Baking, Frozen Pizza & Pizza Snacks"
1,12 Pack OR 2 Liter AND Whole Pizza at Casey's,CASEYS GENERAL STORE,CASEYS GENERAL STORE,"Cooking & Baking, Frozen Pizza & Pizza Snacks"
2,12 pack OR 2 liter AND Whole Pizza at Casey's,CASEYS GENERAL STORE,CASEYS GENERAL STORE,"Cooking & Baking, Frozen Pizza & Pizza Snacks"
3,2 Pack OR 2 Liter AND Whole Pizza at Casey's,CASEYS GENERAL STORE,CASEYS GENERAL STORE,"Cooking & Baking, Frozen Pizza & Pizza Snacks"
4,:ratio™ KETO* Friendly Cereal OR Granola,,RATIO,"Yogurt, Cereal, Granola, & Toaster Pastries, N..."


### Embedding Model

We used Sentence-Transformers pipeline to extract the dense representation of different entities.

The base is `msmarco-distilbert-base-v4` which is trained for asymmetric semantic search where the query is short but documents are large which suits our usecase very well.

In [5]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('msmarco-distilbert-base-v4')

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [6]:
def embed_input(inputs):
  embeddings = model.encode(inputs)
  embeddings /= np.expand_dims(np.linalg.norm(embeddings, ord=2, axis=-1), axis=1)
  return embeddings

### Feature extraction and Vector Store

We extracted dense feature extraction for each columns separately.

For each entity we created separated index for enabling fast and efficient retrieval and search

We used **FAISS** Approximate Nearest Neighbors Search tool that creates an index of dense representation in another space and given another dense representation can efficienty retrieve nearest embedding from the index.

In [39]:
lists = {
    # 'parent_category':offers_df['IS_CHILD_CATEGORY_TO'].tolist(),
    'category' : offers_df['PRODUCT_CATEGORY'].tolist(),
    'brand' : offers_df['BRAND'].tolist(),
    'retailer' : offers_df['RETAILER'].tolist(),
    'offer' : offers_df['OFFER'].tolist()
    }

embeddings = {}
indexes = {}
for name, l in lists.items():
  embeddings[name] = embed_input(l)
  indexes[name] = faiss.IndexFlatL2(embeddings[name].shape[1])
  indexes[name].add(embeddings[name])

## Retrieval Pipeline

Users can query for any brand, retailer, and product category using natural langauge text. Sentence Transformers take the mean of individual tokens in a sentence and return theb final embedding. This could lead to loss of information with respect to some important keywords. For example, if an offer is described by a really long sentence that contains a keyword like `kroger`, the information about the keyword would get lost in the embedding due to averaging. Therefore, we created 4 different indexes.

**Pipeline**: Given a query embedding

1. If it is really close (distance < 1) to an entry in brand embedding, or retailer embedding or a category embedding we retrieve the corresponding offer.
2. If it is close to an offer embedding (distance < 1.25), we retrieve the offer.

Note: These thresholds are empirically set for the given dataset.

## Scoring of retrieved Offers

We retrieve the offers that are closest to the query in the embedding space. Given the pipeline, our scores range from $10^{-10}$ (when query matches brand, or retailer) to $10$ (query matches a keyword in an offer).

To keep the scores in a limited range, we propose the following score function

$$\begin{align*}
logit = \min (100, 1/dist) \\
score = \log(1 + logit)
\end{align*}$$

First, we clipped the logit between 0 and 100, then take the log to scale it down further. 1 added to log to keep the arguments more than 1 and hence keep the scores positive.

Note that multiple retrieved offers can have same scores.

In [58]:
pd.set_option('display.max_colwidth', None)

def get_offers(query):
  query_embedding = embed_input([query])

  # search_order = ['brand', 'retailer', 'category', 'parent_category', 'offer']
  search_order = ['brand', 'retailer', 'category', 'offer']
  results = {}
  for name in search_order:
    index = indexes[name]
    dists, indices = index.search(query_embedding, 10)
    results[name] = []
    for idx, dist in zip(indices[0], dists[0]):
      if idx>=0:
        matched_phrase = lists[name][idx]
        if matched_phrase == '':
          continue
        results[name].append((matched_phrase, dist, idx))

  retrieved_indices = set()
  scores = {}

  for name in search_order:
    # print('name:', name)
    for phrase, dist, idx in results[name]:
      if dist < 1:
        retrieved_indices.add(idx)
        if idx not in scores.keys():
          scores[idx] = min(1/dist, 100)
          scores[idx] = np.log(scores[idx]+1)
        # print(phrase,'|', lists['offer'][idx], '|', dist)
    # print()

  # print('name: offer')
  for phrase, dist, idx in results['offer']:
    if dist < 1.25:
      retrieved_indices.add(idx)
      if idx not in scores.keys():
        scores[idx] = min(1/dist, 100)
        scores[idx] = np.log(1+scores[idx])
      # print(phrase,'|', '|', dist)
  # print()

  retrieved_indices = list(retrieved_indices)
  score_col = [(idx, scores[idx]) for idx in retrieved_indices]
  # print(score_col)
  score_col.sort(key=lambda x : x[1], reverse=True)
  # print(score_col)
  retrieved_indices = [idx for idx, score in score_col]
  scores = [score for idx, score in score_col]
  retrieved_offers = offers_df.iloc[retrieved_indices]
  # retrieved_offers = retrieved_offers.drop('IS_CHILD_CATEGORY_TO', axis=1)
  retrieved_offers['SCORE'] = scores

  # print_outputs(retrieved_offers)
  return retrieved_offers

def print_outputs(output_df):
  if len(output_df) == 0:
    print('No Offers :(')
  else:
    output_df = output_df.reset_index(drop=True)
    output_df.index += 1
    print(output_df)


## Testing the Engine

**Now let's make some queries!**

In [59]:
print_outputs(get_offers('bread')) # product category

                                                         OFFER RETAILER  \
1                                       The Rustik Oven® bread            
2                                        Bays® English Muffins            
3                                        BallPark® buns, buy 2            
4                  Sara Lee® or Alfaros® Artesano bread, buy 2            
5                  Sara Lee® or Alfaros® Artesano bread, buy 5            
6               Sara Lee® or Alfaros® Artesano bread, spend $8            
7              Sara Lee® or Alfaros® Artesano bread, spend $20            
8                            Artesano® buns, buy 2 at Walmart®  WALMART   
9               Arnold, Brownberry, Oroweat® Keto bread, buy 2            
10  Arnold, Brownberry, Oroweat® Small Slice bread, at Walmart  WALMART   
11                           Sara Lee® Delightful bread, buy 2            

                        BRAND  \
1                 RUSTIK OVEN   
2                        BAYS   


<ipython-input-58-ab642e82e110>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retrieved_offers['SCORE'] = scores


In [60]:
print_outputs(get_offers('walmart')) ## Retailer

                                                                     OFFER  \
1   Back to the Roots Grow Hydroponic Grow Kit OR Refill Bundle at Walmart   
2                 Back to the Roots Grow Kits at Walmart or The Home Depot   
3         Back to the Roots Microgreens Grow Kit OR Seed Refill at Walmart   
4                                                       AleveX™ at Walmart   
5                Aleve® products AND AleveX™, select varieties, at Walmart   
6                  Aleve® products, select varieties, 80 count+ at Walmart   
7                                      Back to the Roots Seeds, at Walmart   
8                                                        Arber, at Walmart   
9               Arnold, Brownberry, Oroweat® Small Slice bread, at Walmart   
10                                       Artesano® buns, buy 2 at Walmart®   
11                                                Spend $15 at Burger King   
12                                     Little Bites® Spend $10 a

<ipython-input-58-ab642e82e110>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retrieved_offers['SCORE'] = scores


In [62]:
print_outputs(get_offers('pepsi')) ## BRAND

                                                                           OFFER  \
1  PepsiCo® Beverage, 7.5-ounce 10 pack, select varieties, at Amazon Storefront*   
2                 PepsiCo® Variety Pack, select varieties, at Amazon Storefront*   

  RETAILER  BRAND        PRODUCT_CATEGORY     SCORE  
1   AMAZON  PEPSI  Carbonated Soft Drinks  4.615121  
2   AMAZON  PEPSI  Carbonated Soft Drinks  4.615121  


<ipython-input-58-ab642e82e110>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retrieved_offers['SCORE'] = scores


In [63]:
'''
Note that only one entry in retailer is kroger but other offers contains the keyword kroger
and our model is able to detect that too.
'''
print_outputs(get_offers('kroger'))

                                                            OFFER  \
1               Butterball, select varieties, spend $10 at Kroger   
2        GATORADE® Fast Twitch®, 12-ounce single serve, at Kroger   
3  GATORADE® Fast Twitch®, 12-ounce single serve, buy 2 at Kroger   
4  GATORADE® Fast Twitch®, 12-ounce single serve, buy 2 at Kroger   
5                   Starry™ Lemon Lime Soda multipacks, at Kroger   

          RETAILER       BRAND  \
1           KROGER  BUTTERBALL   
2  DILLONS GROCERY    GATORADE   
3        FOOD4LESS    GATORADE   
4       FRED MEYER    GATORADE   
5  DILLONS GROCERY      STARRY   

                                                    PRODUCT_CATEGORY     SCORE  
1                                   Nut Butters & Jam, Frozen Turkey  4.615121  
2  Sports Drinks, Medicines & Treatments, Meal Replacement Beverages  0.647660  
3  Sports Drinks, Medicines & Treatments, Meal Replacement Beverages  0.645333  
4  Sports Drinks, Medicines & Treatments, Meal Replaceme

<ipython-input-58-ab642e82e110>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retrieved_offers['SCORE'] = scores
